<a href="https://colab.research.google.com/github/Rheddes/RecoveryJournal/blob/master/recsys_twitter_make_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Necessary imports & definitions

Copy files from drive to local disk, not necessary it is also possible to work directly from drive.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Get validation set

Get the validation/prediction set from the challenge.

Set train file variable to correct path

In [0]:
# train_file = './sample.tsv'
# train_features_file = './bert_22500.csv'
# pred_file = ./val.tsv

# OR IF USING THE ONE FROM DRIVE
train_file = './drive/My Drive/RecSys/sample.tsv'
train_features_file = './drive/My Drive/RecSys/bert_22500.csv'
pred_file = './drive/My Drive/RecSys/val.tsv'
pred_features_file = './drive/My Drive/RecSys/bert_22500.csv' # CHANGE WITH ACTUAL FEATURE FILE OF VALIDATION SET

## Imports

In [0]:
import pandas as pd
import numpy as np
import csv
import math
import gc
import torch
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from itertools import islice
import pickle

# Read dataset

Read labels & features from csv files

In [3]:
all_features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",
                "tweet_type", "language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",
                "engaged_with_user_following_count", "engaged_with_user_is_verified",
                "engaged_with_user_account_creation", "enaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count",
                "enaging_user_is_verified", "enaging_user_account_creation", "engagee_follows_engager"]
label_names = ["replied", "retweeted", "retweeted_with_comment", "liked"]

# IF FEATURES AS CSV
features = pd.read_csv(train_features_file).values
# IF FEATURES AS PICKLE OBJECT
# features = pickle.load(open(feature_file, 'rb'))

labels = pd.read_csv(train_file, header=None, sep="\x01")
labels.columns = all_features + label_names
labels = labels[['replied','retweeted', 'retweeted_with_comment', 'liked']]

print("converting now")

for label_name in label_names:
  labels[label_name] = labels[label_name].apply(lambda x: 0 if math.isnan(x) else 1)

# IF NOT ALL SAMPLES HAVE BEEN CONVERTED TO BERT FEATURES
labels = labels[:len(features)]


converting now


In [0]:
print(labels)

In [0]:
validation = pd.read_csv(pred_file, header=None, sep="\x01")
validation.columns = all_features

# IF FEATURES AS CSV
validation_features = pd.read_csv(pred_features_file).values
# IF FEATURES AS PICKLE OBJECT
# validation_features = pickle.load(open(pred_features_file, 'rb'))

# IF NOT ALL ROWS HAVE FEATURES
validation = validation[:len(validation_features)]

In [0]:
print(validation)

# Train model

We got our output from the BERT model we can now train our logistics classifier to actually classify tweet engagements.

## Dimensionality Reduction

Maybe not needed

In [0]:
from sklearn.decomposition import PCA, IncrementalPCA

use_pca = True

if use_pca:
  n_components = 20
  ipca = IncrementalPCA(n_components=n_components, batch_size=150)
  train_features = ipca.fit_transform(features)
else:
  train_features = features

## Train Model on test features

Next train our Logistics Classifier

In [21]:
from sklearn.svm import LinearSVC


models = {}

for label_name in label_names:
  models[label_name] = LinearSVC(C=1.0)
  models[label_name].fit(train_features, labels[label_name].values)

print(models)

[[ 0.00000000e+00  1.74062130e-01 -4.33971060e-02 ...  5.50689700e-01
   1.86707590e-01 -1.98577990e-01]
 [ 1.00000000e+00 -1.19644050e-01 -2.67190340e-01 ...  3.06257670e-01
   3.19380570e-02 -4.96699360e-02]
 [ 2.00000000e+00  1.01058975e-01 -1.48502190e-02 ...  2.96512500e-01
   2.80769140e-01  3.42546960e-02]
 ...
 [ 2.24997000e+05 -2.54571470e-01  1.40958280e-01 ...  4.86173000e-01
   4.11130560e-02 -9.20219900e-02]
 [ 2.24998000e+05 -9.26982400e-03 -1.80375170e-01 ...  2.18384500e-01
   1.56419380e-01 -1.52135860e-01]
 [ 2.24999000e+05 -3.41064630e-01  4.65842400e-02 ...  3.48190160e-01
   8.76392950e-02 -8.85488100e-02]]
[0 0 0 ... 0 0 0]
[[ 0.00000000e+00  1.74062130e-01 -4.33971060e-02 ...  5.50689700e-01
   1.86707590e-01 -1.98577990e-01]
 [ 1.00000000e+00 -1.19644050e-01 -2.67190340e-01 ...  3.06257670e-01
   3.19380570e-02 -4.96699360e-02]
 [ 2.00000000e+00  1.01058975e-01 -1.48502190e-02 ...  2.96512500e-01
   2.80769140e-01  3.42546960e-02]
 ...
 [ 2.24997000e+05 -2.54571

# Create predictions

Now that we have a trained model, we can make predictions for the validation set.

## If dimensionality reduction

Perform PCA transform if dimensionality reduction was used.

In [0]:
if use_pca:
  validation_features = ipca.transform(validation_features)

## Create Predictions

Now we can actually generate predictions from our model.

In [0]:
for label_name, classifier in models.items():
  predictions = classifier.predict(validation_features)
  validation[label_name + '_prediction'] = predictions

In [0]:
!mkdir predictions

for label_name in label_names:
  prediction_column = label_name + '_prediction'
  prediction = validation[['tweet_id', 'enaging_user_id', prediction_column]]
  prediction.to_csv('./predictions/{}.csv'.format(prediction_column), header=None, index=None)

In [16]:
validation.retweeted_with_comment_prediction.value_counts()

0    225000
Name: retweeted_with_comment_prediction, dtype: int64

In [11]:
i = 0
with open('./predictions/replied_prediction.csv', encoding="utf-8") as fileobject:
    for line in fileobject:
      print(line)
      if i > 10:
        break
      i += 1

7647B4E9DAF4C1D8973397DC2A04F3E3,0000006C3074607050F1339DDCB890BB,0

408DB1803264B5FF55F73EC06BE9BD77,000013315386492275CCBF7AEF293EF0,0

2EE951379C47E8BF62EABB8FA027F753,00001569CB28972FC8173122D9DA162F,0

2135F24B05DAE3EF213F9CE80FDC6DAF,00001607209C5774DF9207A2AC0EED5F,0

09143FEDE9BD494A6EA9A7EE160565E3,0000177705514C315F2FC6DFA3872712,0

60968762145D2AF58A58AFB376B2B00C,00001BC70532632181F17B2A65EFD2BA,0

3487905D0C69B0FE4A6C7A301FB6F8BF,00001D9D15FBADE90ADE2577DC0C3593,0

706310D7975C15B9FB1FA3FBAE8A126B,00001F56CDCF81D2EF635B3C0EDE57EB,0

DBC37B8C8DC70C70F588D37CC5006ABB,00001F56CDCF81D2EF635B3C0EDE57EB,0

BA7917AA4B620B13264A68F455203934,0000376314CAC0A3E9D4FCF4A29004D6,0

60DD856C81BC0A115365FA556672C2DB,000043D9A730DF47697D0750F509B56A,0

CFBD0716FB1FE99692156EC937E598B7,00004E42009644A7647E8C988C072D9D,0

